In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_6.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,Most of the time when a student comes back fro...,0.0
1,Car usage all over the world has reduced and b...,0.0
2,Citizens car are good to have but all the fine...,0.0
3,Having a car now a day bib very important and ...,0.0
4,"With the help of technology, society now has a...",0.0
...,...,...
17983,I am definitely against this. For years the po...,0.0
17984,Not many can imagine life without a car. Yet d...,0.0
17985,Will students learn more if they design their ...,0.0
17986,"Dear, Mr Senator\n\nMy name is PROPER_NAME, an...",0.0


In [4]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [5]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1958])
Attention Mask size: torch.Size([17988, 1958])
Vocabulary Size: 30352
Max Sequence Length: 1958


In [6]:
model=Sequential()
input_length=1958
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=128,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)



Epoch 1/5


450/450 [==============================] - 282s 621ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 274s 609ms/step - loss: 5.0111e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 274s 608ms/step - loss: 2.5778e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 275s 611ms/step - loss: 1.5780e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 859s 2s/step - loss: 1.0539e-05 - accuracy: 1.0000


In [7]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 59s 515ms/step - loss: 8.7314e-06 - accuracy: 1.0000


[8.731443813303486e-06, 1.0]

In [8]:
model.save("newmodel6.keras")